In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [175]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [176]:
prep_b=df_ratings.groupby("isbn")["rating"].count().reset_index()
book_lst=(prep_b[prep_b["rating"]>=100]["isbn"]).tolist()
sub_books=df_books[df_books["isbn"].isin(book_lst)]

prep_u=df_ratings[["user","rating"]].groupby("user").count().reset_index()
user_lst=(prep_u[prep_u["rating"]>=200]["user"]).tolist()
sub_ratings=df_ratings[df_ratings["user"].isin(user_lst)]

df=sub_ratings[sub_ratings["isbn"].isin(sub_books["isbn"])]

In [177]:
df_book_features=df.pivot(index="isbn",columns="user",values="rating").fillna(0)

book_to_idx = {
book: i for i, book in
enumerate(list(df_books.set_index('isbn').loc[df_book_features.index].title))
}
idx_to_book = dict((v,k) for k,v in book_to_idx.items())

mat_book_features=csr_matrix(df_book_features.values)

In [178]:
model_knn = NearestNeighbors(metric='cosine')
model_knn.fit(mat_book_features)

NearestNeighbors(metric='cosine')

In [179]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  fav_lst=[book]
  bk_id=book_to_idx[book]
  distances, indices = model_knn.kneighbors(mat_book_features[bk_id], n_neighbors=6)
  d=list(distances[0][1:])
  ind=list(indices[0][1:])
  hlp={}
  for i in range(5):
     hlp[d[i]]=ind[i]
  rev=[]
  for i in range(5):
     rev.append(d[4-i])
  tt=[]
  for i in range(5):
     tt.append(idx_to_book[hlp[rev[i]]])
  fav_lst.append([[],[],[],[],[]])
  for i in range(5):
     fav_lst[1][i].append(tt[i])
     fav_lst[1][i].append(rev[i])  
  recommended_books=fav_lst  
  return recommended_books

In [180]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
